In [106]:
import numpy as np
import holoviews as hv
from holoviews import opts
import xarray as xr
import rioxarray as rx
import hvplot.xarray
import pyproj
import pangaea as pa
import rasterio
import rasterio.features
import rasterio.warp
import hvplot.xarray
import holoviews as hv
import geoviews as gv
import panel as pn
from math import ceil

In [107]:
help(pa.xlsm.LSMGridReader.to_utm)

Help on function to_utm in module pangaea.xlsm:

to_utm(self, variable)
    Convert Grid to UTM projection at center of grid.
    
    Parameters
    ----------
    variable: :obj:`str`
        Name of variable in dataset.
    
    Returns
    -------
    :func:`xarray.Dataset`



In [108]:
data = xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif")[0,:,:]

In [109]:
data = data.where(data>0)

In [110]:
data.name = 'originial'

In [111]:
merge = xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w066/USGS_1_n19w066.tif")[0,:,:]

In [112]:
merge.name = 'other'

In [113]:
merge = merge.where(merge>0)

In [114]:
#test = data.loc[dict(y=slice(-65,-65.1), x = slice(18,19))]
test = data.where(data.x<-64.8258)

In [129]:
list(range(4))

[0, 1, 2, 3]

In [115]:
def createlink(x,y):
    global link
    link = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/'
    north = "{:02d}".format(round(abs(y)))
    west =  "{:03d}".format(round(abs(x)))
    coordinates = 'n'+north+'w'+west
    finallink = link+coordinates+'/USGS_1_'+coordinates+'.tif'
    link = finallink
    return (finallink,coordinates) 

In [116]:
def getregions(startpoint,endpoint):
    #NOTE: this only works if the startpoint is below the end point. Remember to compensate for that in code.
    start = (ceil(float(startpoint[0])),ceil(float(startpoint[1])))
    if float(endpoint[1]).is_integer():
        end = (ceil(endpoint[0]),ceil(endpoint[1]))
    else:
        end = (ceil(endpoint[0])+1,ceil(endpoint[1]))
    regions = []
    for horiz in range(start[0],end[0]):
        for vert in range(start[1]+1,end[1]+1):
            regions.append((horiz,vert))
    return regions

In [147]:
def getpoints(startpoint,endpoint):
    xs = [startpoint[0],endpoint[0]]
    xs+=list(range(ceil(startpoint[0]),ceil(endpoint[0])))
    ys = [startpoint[1],endpoint[1]]
    ys+=list(range(ceil(startpoint[1]),ceil(endpoint[1])))
    points = []
    for x in xs:
        for y in ys:
            points.append((x,y))
    return points

In [127]:
def getregiondata(points):
    #function must be given the top left corner of each region
    regions = []
    for p in points:
        try:
            place = createlink(p[0],p[1])[0]
            data = rx.open_rasterio(place, masked=True).squeeze().astype('float64')
            data.attrs = xr.open_rasterio(place).attrs
            data = data.where(data>0)
            regions.append(data)
        except:
            print('point '+str(p)+' is not avaliable')
    return regions

In [149]:
def bounddata(startpoint,endpoint):
    regions = getregions(startpoint,endpoint)
    unprocessed = getregiondata(regions)
    points = getpoints(startpoint,endpoint)
    for r in unprocessed:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-149-df023027183d>, line 6)

In [122]:
regions=getregions((96,53),(98,54))

In [123]:
regions

[(96, 54), (97, 54)]

In [125]:
getregiondata(regions)

2

In [26]:
n54w096=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n53w099/USGS_1_n53w099.tif").squeeze()
n54w097=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n54w097/USGS_1_n54w097.tif").squeeze()
n55w097=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n55w097/USGS_1_n55w097.tif").squeeze()
n55w096=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n55w096/USGS_1_n55w096.tif").squeeze()

In [27]:
n54w096.name='test'
n54w097.name='test'
n55w097.name='test'
n55w096.name='test'

In [29]:
large = xr.combine_nested([[n54w097],[n55w096],[n55w097],[n54w096]],concat_dim=['x',None])


In [33]:
def combineregions(data):
    regions = []
    for region in data:
        region.name = 'part'
        regions.append([region])
    combination = xr.combine_nested(regions,concat_dim=['x',None])
    #fix this line the attr are not correct
    combination.attrs = data[0].attrs
    return combination

In [34]:
test= combineregions([n54w096,n54w097,n55w097,n55w096])

In [35]:
test

<xarray.Dataset>
Dimensions:  (x: 14448, y: 10836)
Coordinates:
    band     int64 1
  * y        (y) float64 52.0 52.0 52.0 52.0 52.0 ... 55.0 55.0 55.0 55.0 55.0
  * x        (x) float64 -99.0 -99.0 -99.0 -99.0 ... -95.0 -95.0 -95.0 -95.0
Data variables:
    part     (y, x) float32 276.4687 276.52182 275.87344 ... 215.31635 215.19669
Attributes:
    transform:      (0.0002777777777780002, 0.0, -99.00166666666667, 0.0, -0....
    crs:            +init=epsg:4269
    res:            (0.0002777777777780002, 0.0002777777777780002)
    is_tiled:       1
    nodatavals:     (-999999.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    descriptions:   ('Layer_1',)
    AREA_OR_POINT:  Area

In [38]:
test.hvplot.quadmesh(rasterize = True, geo = True, tiles = 'OSM', cmap = 'inferno', alpha = 20, frame_width=300, frame_height=300)

/home/jovyan/my-conda-envs/dem/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/home/jovyan/my-conda-envs/dem/lib/python3.8/site-packages/holoviews/plotting/util.py:685: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  [cmap for cmap in cm.cmap_d if not
/home/jovyan/my-conda-envs/dem/lib/python3.8/site-packages/holoviews/plotting/util.py:685: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  [cmap for cmap in cm.cmap_d if not


:DynamicMap   []
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .Image.I :Image   [x,y]   (part)

In [ ]:
def 

In [25]:
import numpy as np
import rasterio
from rasterio import Affine as A
from rasterio.warp import reproject, Resampling

with rasterio.Env():

    # As source: a 512 x 512 raster centered on 0 degrees E and 0
    # degrees N, each pixel covering 15".
    rows, cols = src_shape = (3612, 3612)
    d = 1/(3612**2) # decimal degrees per pixel
    # The following is equivalent to
    # A(d, 0, -cols*d/2, 0, -d, rows*d/2).
    src_transform = A.translation(-cols*d/2, rows*d/2) * A.scale(1,)
    src_crs = {'init': 'EPSG:4269'}
    source = data

    # Destination: a 1024 x 1024 dataset in Web Mercator (EPSG:3857)
    # with origin at 0.0, 0.0.
    dst_shape = (3612, 3612)
    dst_transform = A.translation(-237481.5, 237536.4) * A.scale(425.0, -425.0)
    dst_crs = {'init': 'EPSG:3857'}
    destination = np.zeros(dst_shape, np.uint8)

    reproject(
        source,
        destination,
        src_transform=src_transform,
        src_crs=src_crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest)


AttributeError: 'DataArray' object has no attribute 'reshape'

In [ ]:
destination

In [ ]:
work = xr.DataArray(destination)

In [ ]:
work = work.where(work>0)